In [ ]:
###inferenssi (mallin testaus.)
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from transformers import TextStreamer

tokenizer = AutoTokenizer.from_pretrained("./mistral-dpo-lora-adapter") #ladataan tokenizer.
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

base_model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.1",
    torch_dtype=torch.bfloat16, #koulutuksessa käytettiin bf16, joten käytetään myös inferenssissä.
    device_map="auto"
)

model = PeftModel.from_pretrained(base_model, "./mistral-dpo-lora-adapter") #ladataan hienosäädetty malli

streamer = TextStreamer( #tämä on sitä varten, että promptin output näkyy reaaliajassa.
    tokenizer, 
    skip_prompt=True,
    skip_special_tokens=True
)


In [ ]:
prompt = "[INST] What is a GPU? [/INST]" #Syöte kielimallille
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

_ = model.generate(
    **inputs,
    max_new_tokens=200,
    streamer=streamer,
    temperature=0.7 #tokenien Tn skaalaus. isompi arvo tuottaa "kaoottisemman" outputin.
)